In [1]:
from dandi.dandiapi import DandiAPIClient

dandiset_id = '000055'  # ephys dataset from the Svoboda Lab
filepath = 'sub-01/sub-01_ses-7_behavior+ecephys.nwb'  # 450 kB file
with DandiAPIClient() as client:
    asset = client.get_dandiset(dandiset_id).get_asset_by_path(filepath)
    s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)

A newer version (0.58.2) of dandi/dandi-cli is available. You are using 0.58.1


In [4]:
import fsspec
import pynwb
import h5py
from fsspec.implementations.cached import CachingFileSystem

# first, create a virtual filesystem based on the http protocol
fs = fsspec.filesystem("http")

# create a cache to save downloaded data to disk (optional)
fs = CachingFileSystem(
    fs=fs,
    cache_storage="nwb-cache",  # Local folder for the cache
)

# next, open the file
f = fs.open(s3_url, "rb")
file = h5py.File(f)
io = pynwb.NWBHDF5IO(file=file, load_namespaces=True)
nwbfile = io.read()
print(nwbfile.processing)

/Users/rly/mambaforge/envs/arrays/lib/python3.11/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.4.0-alpha because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/rly/mambaforge/envs/arrays/lib/python3.11/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.2.5 because version 2.6.0-alpha is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/Users/rly/mambaforge/envs/arrays/lib/python3.11/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


{'behavior': behavior pynwb.base.ProcessingModule at 0x6021083152
Fields:
  data_interfaces: {
    Position <class 'pynwb.behavior.Position'>,
    ReachEvents <class 'abc.Events'>
  }
  description: pose data
}


In [44]:
l_ear = nwbfile.processing["behavior"]["Position"]["L_Ear"]
print(l_ear.data.shape)
print(l_ear.rate)
# some files have non-nan data here...


(2592000, 2)
30.0


In [50]:
nwbfile.processing["behavior"]["ReachEvents"]

ReachEvents abc.Events at 0x6020983888
Fields:
  description: r_wrist
  timestamps: <HDF5 dataset "timestamps": shape (190,), type "<f8">
  timestamps__resolution: 0.002
  timestamps__unit: seconds

In [45]:
# there are 190 reaches and timestamps start at 1630.97
print(len(nwbfile.processing["behavior"]["ReachEvents"].timestamps))
nwbfile.processing["behavior"]["ReachEvents"].timestamps[:]

190


array([ 1630.97 ,  1641.204,  2118.256,  2360.698,  2430.918,  2439.552,
        2612.638,  2844.218,  3215.422,  3563.386,  3626.738,  3683.806,
        4184.924,  4205.424,  4863.428,  4877.562,  5172.968,  5346.502,
        5594.688,  5774.93 ,  6423.24 ,  7137.94 ,  7151.372, 11997.632,
       16713.078, 16840.284, 16848.952, 16864.484, 17067.548, 17150.982,
       18544.596, 18611.528, 19473.082, 19477.348, 19493.116, 20476.814,
       21312.074, 21538.15 , 21554.582, 22329.738, 22833.688, 22921.688,
       23104.228, 23112.496, 23234.77 , 23258.402, 23519.008, 23522.04 ,
       23644.224, 23882.398, 24058.916, 24290.09 , 24666.558, 24704.458,
       24794.888, 24805.922, 24847.488, 24886.064, 25118.506, 25511.408,
       25566.274, 25718.348, 25978.   , 26070.088, 26074.388, 26702.638,
       27004.604, 27470.792, 28466.358, 28517.758, 30202.626, 30307.146,
       30524.932, 31104.602, 31146.434, 31772.976, 32257.8  , 32260.7  ,
       32402.702, 32438.004, 32507.504, 32515.972, 

In [46]:
# there are 190 rows of the reaches table and start_time values start at 32313.385...
nwbfile.intervals["reaches"].to_dataframe()

,start_time,stop_time,Reach_magnitude_px,Reach_angle_degrees,Onset_speed_px_per_sec,Speech_ratio,Bimanual_ratio,Bimanual_overlap,Bimanual_class
id,,,,,,,,,
0,32313.385,32313.718333,86.550017,-94.917874,360.145666,0.388753,0.007263,0.000000,0.0
1,32323.618,32323.951333,84.996598,-105.331056,357.631204,0.362544,0.358579,0.266667,1.0
2,32800.671,32801.337667,48.191307,-97.642073,108.916704,0.499110,0.034265,0.000000,0.0
3,33043.112,33043.445333,95.467849,46.594854,398.035815,0.653675,0.001902,0.000000,0.0
4,33113.332,33117.165333,120.785173,-73.574682,221.310953,0.561869,0.414411,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
185,84975.511,84976.277667,129.912785,-64.710374,116.456616,0.098857,0.027929,0.000000,0.0
186,85350.773,85352.773000,201.857573,104.471622,296.779466,0.167435,1.000000,0.533333,1.0
187,85401.139,85401.872333,130.318884,5.756270,194.096659,0.180105,0.007709,0.000000,1.0


In [20]:
# compute the offset between the first values
offset = nwbfile.intervals["reaches"][0, "start_time"] - nwbfile.processing["behavior"]["ReachEvents"].timestamps[0]
offset

30682.414999999997

In [26]:
# apply the offset to the timestamps and compare with the start_time values
# the values are basically the same, but there are some rounding errors
import numpy as np
np.round(nwbfile.intervals["reaches"]["start_time"][:] - (nwbfile.processing["behavior"]["ReachEvents"].timestamps + offset), 3)

array([ 0.   , -0.001,  0.   , -0.001, -0.001, -0.001,  0.   ,  0.   ,
       -0.001, -0.001,  0.   , -0.001,  0.   ,  0.   , -0.001, -0.001,
        0.   , -0.001,  0.   , -0.001,  0.   , -0.001,  0.   ,  0.   ,
       -0.001,  0.   , -0.001,  0.   ,  0.   , -0.001, -0.001,  0.   ,
       -0.001,  0.   , -0.002,  0.   ,  0.   , -0.001,  0.   ,  0.   ,
       -0.001, -0.001,  0.   , -0.001, -0.001,  0.   , -0.001,  0.   ,
       -0.001, -0.001, -0.001,  0.   ,  0.   ,  0.   ,  0.   , -0.001,
        0.   ,  0.   , -0.001, -0.001,  0.   , -0.002,  0.   ,  0.   ,
        0.   ,  0.   , -0.001, -0.001,  0.   ,  0.   ,  0.   , -0.   ,
       -0.001, -0.001,  0.   , -0.   ,  0.   ,  0.   , -0.001,  0.   ,
        0.   , -0.001, -0.001,  0.   ,  0.   ,  0.   ,  0.   , -0.001,
        0.   ,  0.   ,  0.   , -0.001, -0.001,  0.   , -0.001,  0.   ,
       -0.001,  0.   , -0.001, -0.001, -0.001, -0.001,  0.   , -0.001,
       -0.001, -0.001, -0.001,  0.   , -0.001,  0.   , -0.001, -0.001,
      

In [49]:
# the epochs table contains times from 0 to 86400 seconds = 24 hours
# and labels for each period within those 24 hours
# the reaches table seems to be in the same clock, ending at 85787.809333 seconds
# the epochs with a label that seems related to the experiment start at 37903.23333 seconds
# this is after the first start time in the reaches table
nwbfile.intervals["epochs"].to_dataframe().query("labels == 'Blocklist (Experiment)'")

,start_time,stop_time,labels
id,,,
118,37903.233333,38013.066667,Blocklist (Experiment)
120,38013.166667,38132.566667,Blocklist (Experiment)
122,38132.666667,38254.733333,Blocklist (Experiment)
124,38254.800000,38372.900000,Blocklist (Experiment)
126,38373.000000,38496.466667,Blocklist (Experiment)
...,...,...,...
250,45813.266667,45934.000000,Blocklist (Experiment)
252,45934.033333,46054.666667,Blocklist (Experiment)
254,46054.700000,46173.000000,Blocklist (Experiment)


In [ ]:
# most likely, the